# MLP Lesion Test

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import sys
sys.path.append('..')

from IPython import display
from tqdm import tqdm
import seaborn as sns
import pandas as pd

from src.lesion import (perform_lesion_experiment,
                        plot_all_damaged_clusters, compute_damaged_cluster_stats,
                        plot_overall_damaged_clusters)
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path

sns.set(style='whitegrid')

In [ ]:
MLP_MODELS = [
              ('mnist', 'MNIST'),
              ('line', 'LINE'),
              ('cifar10', 'CIFAR10'),
              ('fashion', 'FASHION'),
              ('random', 'RANDOM'),
              ('mnist', 'MNIST+DROPOUT'),
              ('line', 'LINE+DROPOUT'),
              ('cifar10', 'CIFAR10+DROPOUT'),
              ('fashion', 'FASHION+DROPOUT')
             ]

In [ ]:
%%time

with pd.ExcelWriter('../results/mlp-lesion.xlsx') as writer:

    for dataset_name, model_name in tqdm(MLP_MODELS):

        (true_results,
         all_random_results,
         metadata,
         evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                       get_model_path(model_name), n_shuffles=100)

        plot_all_damaged_clusters(true_results, all_random_results, metadata, evaluation,
                                  title=f'{model_name}');

        plot_overall_damaged_clusters(true_results, all_random_results, metadata, evaluation,
                                      title=f'{model_name}');

        df = compute_damaged_cluster_stats(true_results, all_random_results, metadata, evaluation)

        print(f'### {model_name}')
        display.display(df)

df.to_csv(f'../results/mlp-lesion-{model_name}.csv')
df.to_excel(writer, sheet_name=model_name)